# Chatbot 

Web-based interface of the chatbot interface: https://gonzalorecio.com/chatbot/robot.html

In [42]:
import requests
URL_INTERFACE = 'https://gonzalorecio.com/chatbot/robot.html'

In [43]:
class ChatbotFace:
    chatbot_mood_API   = 'https://chatbot-mood.herokuapp.com/mood'
    chatbot_status_API = 'https://chatbot-mood.herokuapp.com/internal_state'

    def change_mood(self, mood):
        req_obj = {'action': mood}
        x = requests.post(self.chatbot_mood_API, data = req_obj)
        # print(x)
        
    def change_status(self, status):
        req_obj = {'status': status}
        x = requests.post(self.chatbot_status_API, data = req_obj)
        
    def neutral(self):
        self.change_mood('neutral')
        
    def happy(self):
        self.change_mood('happy')
        
    def sad(self):
        self.change_mood('sad')
        
    def angry(self):
        self.change_mood('angry')
        
    def focused(self):
        self.change_mood('focused')
        
    def confused(self):
        self.change_mood('confused')
    
    def start_blinking(self):
        self.change_mood('start_blinking')
        
    def stop_blinking(self):
        self.change_mood('stopt_blinking')
        
    def status_listening(self):
        self.change_status('listening')
        
    def status_thinking(self):
        self.change_status('thinking')
    
    def status_recognizing(self):
        self.change_status('Recognizing')
    
    def status_custom(self, text):
        self.change_status(text)
    
    def status_none(self):
        self.change_status('')

In [45]:
chatbot = ChatbotFace()

In [28]:
chatbot.confused()

In [29]:
chatbot.happy()

In [30]:
chatbot.status_thinking()

In [31]:
chatbot.status_listening()

In [46]:
chatbot.status_custom('I\'m pretty fine, how are you doing?')

In [33]:
chatbot.status_none()

In [34]:
chatbot.neutral()

# Sentiment

In [35]:
from sentiments import sentiment_analysis as sent

In [36]:
# Test
sent.sentiment_analysis("I've had a nice day today")

Score: {'neg': 0.0, 'neu': 0.588, 'pos': 0.412, 'compound': 0.4215}
I've had a nice day today
Positive


1

## Speech

In [37]:
from speech import google_speech as speech

In [13]:
# Test
speech.speech_to_audio(lang="en-US")

Listening...
Recognizing...
hola hola


'hola hola'

In [38]:
# Test
s = 'what are you saying?? :D'
speech.text_to_speech(s, speed=150, voice=1)

## Translation

In [12]:
from google_trans_new import google_translator  
translator = google_translator()  
translate_text = translator.translate('สวัสดีจีน', lang_tgt='es')  
print(translate_text)
#output: Hello china

Hola china 


# Chatbot

In [39]:
import tensorflow
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from google_trans_new import google_translator  

class Chatbot:
    def __init__(self, lang='en'):
        print('Loading model and tokenizers...')
        self.model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
        self.tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
        self.face = ChatbotFace()
        
        self.reset_chatbot(lang=lang)
        print('Chatbot ready.')
    
    def reset_chatbot(self, lang='en'):
        self.chat_history_ids = self.tokenizer.encode(self.tokenizer.bos_token, return_tensors='pt')
        if lang=='es':
            self.voice = 0 # 0: spanish female, 1: english female, 2: english male
        else:
            self.voice = 1 # 0: spanish female, 1: english female, 2: english male
        self.translator = google_translator()  
        self.lang = lang
        
    def listen_and_get_question(self):
        self.face.status_listening()
        lang = 'en-US' if self.lang=='en' else 'es-ES'
        question = speech.speech_to_audio(chatbot_face=self.face)
        if self.lang=='es':
            question = self.translate(question, lang='en')
        self.face.status_none()
        return question
    
    def decode(self, token_ids):
        return self.tokenizer.decode(token_ids, skip_special_tokens=True)
    
    def generate_answer(self, question):
        self.face.status_thinking()
        question_ids = self.tokenizer.encode(question + self.tokenizer.eos_token, return_tensors='pt')
        input_ids = torch.cat([self.chat_history_ids, question_ids], dim=-1)
        self.chat_history_ids = self.model.generate(input_ids, max_length=1000, pad_token_id=self.tokenizer.eos_token_id)
        answer_ids = self.chat_history_ids[:, input_ids.shape[-1]:][0]
        text = self.decode(answer_ids)
        if self.lang == 'es':
            text = self.translate(text, lang='es')
        self.face.status_none()
        return text
    
    def get_sentiment_analysis(self, text):
        return True
        
    def speak(self, text):
        self.face.happy()
        speech.text_to_speech(text, speed=150, voice=self.voice)
        self.face.neutral()
        
    def translate(self, text, lang):
        translated_text = self.translator.translate(text, lang_tgt=lang)
        if type(translated_text) == list:
            return translated_text[0]
        return translated_text
    
    def no_understand(self):
        text = "Sorry, I couldn't understand you"
        if self.lang == 'es':
            text = self.translete(text, lang='es')
        self.face.status_custom(text)
        self.speak(text)
    
    def run_chat(self):
        self.reset_chatbot(lang=self.lang)
        self.face.neutral()
        question = ''
        while(question != 'goodbye'):
            question = self.listen_and_get_question()
            if question is None:
                print('Fallo')
                self.no_understand()
                continue
            print('Question:', question)
            answer = self.generate_answer(question)
            print('Answer:', answer)
            self.face.status_custom(answer)
            self.speak(answer)
            
#             self.reset_chatbot(lang=self.lang)
            print(self.chat_history_ids)
        

In [40]:
chatbot = Chatbot(lang='es')
# chatbot.run()

Loading model and tokenizers...
Chatbot ready.


In [41]:
import webbrowser as web

web.open(URL_INTERFACE)

True

In [21]:
chatbot.reset_chatbot(lang='en')

In [23]:
chatbot.run_chat()

Listening...
Recognizing...
how are you
Question: how are you
Answer: I'm good, how are you?
tensor([[50256,  4919,   389,   345, 50256,    40,  1101,   922,   837,   703,
           389,   345,  5633, 50256]])
Listening...
Recognizing...
you are the worst robot ever
Question: you are the worst robot ever
Answer: I'm not a robot
tensor([[50256,  4919,   389,   345, 50256,    40,  1101,   922,   837,   703,
           389,   345,  5633, 50256,  5832,   389,   262,  5290,  9379,  1683,
         50256,    40,  1101,   407,   257,  9379, 50256]])
Listening...
Recognizing...
bona nit
Question: bona nit
Answer: I'm not a bot
tensor([[50256,  4919,   389,   345, 50256,    40,  1101,   922,   837,   703,
           389,   345,  5633, 50256,  5832,   389,   262,  5290,  9379,  1683,
         50256,    40,  1101,   407,   257,  9379, 50256,  4189,    64, 13759,
         50256,    40,  1101,   407,   257, 10214, 50256]])
Listening...
Recognizing...
Could not understand audio
Fallo
Listening...
Re

KeyboardInterrupt: 